# NL to SQL Bot

In [ ]:
pip install openai


In [3]:
import sqlite3


In [ ]:
!pip install --upgrade --force-reinstall pydantic pydantic-core


In [50]:
import sqlite3

conn = sqlite3.connect('travel2.sqlite')
cursor = conn.cursor()

cursor.execute("SELECT DISTINCT departure_airport FROM flights;")
dep_codes = cursor.fetchall()

cursor.execute("SELECT DISTINCT arrival_airport FROM flights;")
arr_codes = cursor.fetchall()

conn.close()

print("Departure Airports:", [code[0] for code in dep_codes])
print("Arrival Airports:", [code[0] for code in arr_codes])


Departure Airports: ['BSL', 'SHA', 'OSL', 'HAM', 'PEK', 'GVA', 'MSP', 'LIS', 'DKR', 'NBO', 'HKT', 'SEA', 'PEN', 'CEB', 'PHX', 'DEN', 'RIX', 'DME', 'DOH', 'SEZ', 'EDI', 'LHR', 'BOM', 'LGA', 'FCO', 'PRG', 'DUS', 'STN', 'SLC', 'HND', 'SYD', 'PER', 'CTU', 'MAD', 'IST', 'GIG', 'IAD', 'BKK', 'IAH', 'LGW', 'BCN', 'MEX', 'SFO', 'MEL', 'AKL', 'KWI', 'GRU', 'JNB', 'CDG', 'CGK', 'BOS', 'HRE', 'CPH', 'PVG', 'KMG', 'ADL', 'PPT', 'SAW', 'JFK', 'DTW', 'SAL', 'MIA', 'CAN', 'ORD', 'CUN', 'LAD', 'LIM', 'LAS', 'ICN', 'SIN', 'REP', 'CLT', 'MRS', 'EWR', 'MUC', 'DEL', 'ATL', 'RGN', 'DUB', 'SVO', 'DCA', 'HKG', 'PNH', 'MNL', 'JRO', 'ATH', 'MCO', 'HEL', 'FRA', 'BRU', 'SAN', 'ORY', 'HNL', 'DFW', 'BNE', 'AMS', 'CCS', 'ARN', 'NGO', 'DXB', 'DAC', 'BOG', 'LAX', 'NRT']
Arrival Airports: ['BKK', 'CUN', 'AMS', 'PRG', 'RGN', 'EDI', 'MNL', 'NBO', 'DEN', 'HNL', 'GVA', 'BOM', 'SHA', 'IAH', 'BOS', 'OSL', 'BSL', 'MAD', 'HKT', 'MUC', 'CDG', 'DUS', 'HAM', 'LHR', 'FCO', 'MSP', 'STN', 'PER', 'CTU', 'IST', 'DKR', 'IAD', 'MEX', '

In [52]:
import sqlite3

def show_city_to_airport_code(db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   
   cursor.execute("SELECT city, airport_code FROM airports_data;")
   mappings = cursor.fetchall()
   conn.close()
   
   print("City to Airport Code mappings:")
   for city, code in mappings:
       print(f"{city} -> {code}")

show_city_to_airport_code()


City to Airport Code mappings:
Atlanta -> ATL
Beijing -> PEK
Dubai -> DXB
Los Angeles -> LAX
Tokyo -> HND
Chicago -> ORD
London -> LHR
Paris -> CDG
Dallas -> DFW
Hong Kong -> HKG
New York -> JFK
Amsterdam -> AMS
Frankfurt -> FRA
Shanghai -> PVG
San Francisco -> SFO
Seoul -> ICN
Denver -> DEN
Bangkok -> BKK
Madrid -> MAD
Singapore -> SIN
Guangzhou -> CAN
Seattle -> SEA
Miami -> MIA
Barcelona -> BCN
Toronto -> YYZ
Phoenix -> PHX
Las Vegas -> LAS
Houston -> IAH
Munich -> MUC
Istanbul -> IST
New Delhi -> DEL
Sydney -> SYD
Orlando -> MCO
Newark -> EWR
London -> LGW
Minneapolis -> MSP
Mumbai -> BOM
Charlotte -> CLT
Kunming -> KMG
Manila -> MNL
Taipei -> TPE
Mexico City -> MEX
Auckland -> AKL
Washington, D.C. -> IAD
São Paulo -> GRU
Rome -> FCO
Chengdu -> CTU
Tampa -> TPA
Boston -> BOS
Calgary -> YYC
Moscow -> DME
Vienna -> VIE
Salt Lake City -> SLC
Düsseldorf -> DUS
Paris -> ORY
Vancouver -> YVR
Johannesburg -> JNB
Zurich -> ZRH
Tokyo -> NRT
New York -> LGA
Brussels -> BRU
Copenhagen -> CPH


In [ ]:
!pip install jiter


In [ ]:
pip show openai


In [ ]:
!pip install --force-reinstall jiter



In [ ]:
import jiter
print(jiter.__version__)


In [51]:
import sqlite3

def show_airports_columns(db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   
   cursor.execute("PRAGMA table_info(airports_data);")
   columns = cursor.fetchall()
   conn.close()
   
   print("Columns in airports_data:")
   for col in columns:
       print(f"- {col[1]}")

show_airports_columns()


Columns in airports_data:
- airport_code
- airport_name
- city
- coordinates
- timezone


In [44]:
import sqlite3

def get_table_columns(table_name, db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   cursor.execute(f"PRAGMA table_info({table_name});")
   columns = cursor.fetchall()
   conn.close()
   return [col[1] for col in columns]  # col[1] is the column name

print("Columns in flights table:", get_table_columns("flights"))


Columns in flights table: ['flight_id', 'flight_no', 'scheduled_departure', 'scheduled_arrival', 'departure_airport', 'arrival_airport', 'status', 'aircraft_code', 'actual_departure', 'actual_arrival']


In [9]:
import sqlite3
import openai

from openai import OpenAI
client = OpenAI(api_key="YOUR_API_KEY_HERE")


In [10]:
def get_table_columns(table_name, db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   cursor.execute(f"PRAGMA table_info({table_name});")
   columns = cursor.fetchall()
   conn.close()
   return [col[1] for col in columns]



In [22]:
def execute_sql(sql_query, db_path="travel2.sqlite", preview_limit=10):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   try:
       cursor.execute(sql_query)
       result = cursor.fetchall()
       conn.close()
       return result[:preview_limit], len(result)
   except Exception as e:
       conn.close()
       return f" Error executing query: {e}", 0


In [23]:
def show_tables(db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
   tables = cursor.fetchall()
   conn.close()
   print(" Tables in DB:", [table[0] for table in tables])


In [41]:
def load_city_to_code_map(db_path="travel2.sqlite"):
   conn = sqlite3.connect(db_path)
   cursor = conn.cursor()
   cursor.execute("SELECT city, airport_code FROM airports_data;")
   data = cursor.fetchall()
   conn.close()
   return {city.lower(): code for city, code in data}


In [46]:
import re

city_code_map = load_city_to_code_map()

def nl_to_sql(nl_query, table_name):
   # Try to handle simple "from <city>" case first with mapping
   user_input_lower = nl_query.lower()
   if "from" in user_input_lower:
       after_from = user_input_lower.split("from")[-1].strip()
       city_name = after_from.split()[0]  # can be improved later for multi-word cities
       airport_code = city_code_map.get(city_name)
       if airport_code:
           return f"SELECT * FROM flights WHERE departure_airport = '{airport_code}';"
   
   # fallback: use GPT to generate SQL
   columns = get_table_columns(table_name)
   prompt = (
       f"You are an expert SQL generator. The table '{table_name}' has columns: {columns}. "
       f"Translate the following natural language request into a SQL query only. "
       f"Do NOT include any explanation or markdown formatting like ```sql. Only output pure SQL."
       f"\nRequest: \"{nl_query}\""
   )

   response = client.chat.completions.create(
       model="gpt-4o",
       messages=[
           {"role": "system", "content": "Translate natural language to SQL ONLY. Return only valid SQL without any explanation."},
           {"role": "user", "content": prompt}
       ]
   )

   raw_output = response.choices[0].message.content.strip()
   clean_sql = re.sub(r"```sql|```", "", raw_output).strip()
   return clean_sql


In [53]:
def chatbot():
   print(" Natural Language to SQL Chatbot (type 'exit' to leave)")
   show_tables()
   
   while True:
       table_name = input("Which table do you want to query? (flights/airports_data/hotels): ").strip()
       user_input = input("\n Your request: ")
       if user_input.lower() == "exit":
           print(" Ended")
           break
       

       sql_query = nl_to_sql(user_input, table_name)
       print(f" GPT's SQL:\n{sql_query}")
       
       result, total = execute_sql(sql_query)
       if isinstance(result, str):  # error
           print(result)
       else:
           print(f" Showing first {len(result)} of {total} results:")
           for row in result:
               print(row)


In [54]:

print("hi")
chatbot()

hi
 Natural Language to SQL Chatbot (type 'exit' to leave)
 Tables in DB: ['aircrafts_data', 'airports_data', 'boarding_passes', 'bookings', 'flights', 'seats', 'ticket_flights', 'tickets', 'car_rentals', 'hotels', 'trip_recommendations']



 Your request:  show all flights from seattle


 GPT's SQL:
SELECT * FROM flights WHERE departure_airport = 'SEA';
 Showing first 10 of 183 results:
(23609, 'TG0055', '2024-05-16 05:44:03.561731-04:00', '2024-05-16 07:09:03.561731-04:00', 'SEA', 'OSL', 'Scheduled', 'CR2', None, None)
(23695, 'DL0114', '2024-05-11 05:04:03.561731-04:00', '2024-05-11 05:34:03.561731-04:00', 'SEA', 'MNL', 'Scheduled', 'CR2', None, None)
(23524, 'IB0115', '2024-04-22 04:19:03.561731-04:00', '2024-04-22 05:39:03.561731-04:00', 'SEA', 'BSL', 'Arrived', 'CR2', '2024-04-22 04:20:03.561731-04:00', '2024-04-22 05:40:03.561731-04:00')
(23525, 'IB0115', '2024-05-14 04:19:03.561731-04:00', '2024-05-14 05:39:03.561731-04:00', 'SEA', 'BSL', 'Scheduled', 'CR2', None, None)
(23526, 'IB0115', '2024-05-23 04:19:03.561731-04:00', '2024-05-23 05:39:03.561731-04:00', 'SEA', 'BSL', 'Scheduled', 'CR2', None, None)
(23527, 'IB0115', '2024-04-19 04:19:03.561731-04:00', '2024-04-19 05:39:03.561731-04:00', 'SEA', 'BSL', 'Arrived', 'CR2', '2024-04-19 04:21:03.561731-04:00', '20


 Your request:  show all flights from paris to tokyo


 GPT's SQL:
SELECT * FROM flights WHERE departure_airport = 'ORY';
 Showing first 10 of 123 results:
(29503, 'CZ0109', '2024-04-21 07:54:03.561731-04:00', '2024-04-21 09:39:03.561731-04:00', 'ORY', 'SHA', 'Arrived', 'CR2', '2024-04-21 08:00:03.561731-04:00', '2024-04-21 09:44:03.561731-04:00')
(29504, 'CZ0109', '2024-04-09 07:54:03.561731-04:00', '2024-04-09 09:39:03.561731-04:00', 'ORY', 'SHA', 'Arrived', 'CR2', '2024-04-09 07:56:03.561731-04:00', '2024-04-09 09:40:03.561731-04:00')
(29505, 'CZ0109', '2024-05-05 07:54:03.561731-04:00', '2024-05-05 09:39:03.561731-04:00', 'ORY', 'SHA', 'Scheduled', 'CR2', None, None)
(29506, 'CZ0109', '2024-03-31 07:54:03.561731-04:00', '2024-03-31 09:39:03.561731-04:00', 'ORY', 'SHA', 'Arrived', 'CR2', '2024-03-31 07:59:03.561731-04:00', '2024-03-31 09:44:03.561731-04:00')
(29507, 'CZ0109', '2024-04-07 07:54:03.561731-04:00', '2024-04-07 09:39:03.561731-04:00', 'ORY', 'SHA', 'Arrived', 'CR2', '2024-04-07 07:58:03.561731-04:00', '2024-04-07 09:44:03.56


 Your request:  exit


 Ended
